# Description - Create Swarm File to run transformation to MNI pipeline on the preprocessed data

This script creates the SWARM file to run the pipeline that will transform the data to MNI space, which is necessary for the following step of this project.

In [6]:
import pandas as pd
import os.path as osp
import os
from datetime import datetime
import getpass
import subprocess

from utils.basics import get_sbj_scan_list

from utils.basics import PRJ_DIR, DATA_DIR, SCRIPTS_DIR #NOTEBOOKS_DIR, RESOURCES_DINFO_DIR, PREPROCESSING_NOTES_DIR, 
print('++ INFO: Project Dir:                  %s' % PRJ_DIR) 
#print('++ INFO: Notebooks Dir:                %s' % NOTEBOOKS_DIR) 
print('++ INFO: Bash Scripts Dir:             %s' % SCRIPTS_DIR)
#print('++ INFO: Resources (Dataset Info) Dir: %s' % RESOURCES_DINFO_DIR)
#print('++ INFO: Pre-processing Notes Dir:     %s' % PREPROCESSING_NOTES_DIR)
print('++ INFO: Data Dir:                     %s' % DATA_DIR)

++ INFO: Project Dir:                  /data/SFIMJGC_Introspec/2023_fc_introspection
++ INFO: Bash Scripts Dir:             /data/SFIMJGC_Introspec/2023_fc_introspection/code/fc_introspection/bash
++ INFO: Data Dir:                     /data/SFIMJGC_Introspec/pdn


In [2]:
username = getpass.getuser()
print('++ INFO: user working now --> %s' % username)

++ INFO: user working now --> javiergc


# 1. Load list of scans that completed struct and func pre-processing and have low motion

In [3]:
sbj_list, scan_list, SNYCQ_data = get_sbj_scan_list('post_motion')

++ [post_motion] Number of subjects: 133 subjects
++ [post_motion] Number of scans:    471 scans


# 2. Create Output Folder for all subjects

In [4]:
for sbj in sbj_list:
    output_path = osp.join(DATA_DIR,'PrcsData',sbj,'preprocessed','func','pb05_mni')
    if not osp.exists(output_path):
        os.makedirs(output_path)

***
# 3. Create SWARM file

This will create a swarm file with one line call per subject. The inputs to that bash script are:

* SBJ = subject ID

In [5]:
#user specific folders
#=====================
swarm_folder   = osp.join(PRJ_DIR,'SwarmFiles.{username}'.format(username=username))
logs_folder    = osp.join(PRJ_DIR,'Logs.{username}'.format(username=username))

swarm_path     = osp.join(swarm_folder,'S04_TransformToMNI.pass01.SWARM.sh')
logdir_path    = osp.join(logs_folder, 'S04_TransformToMNI.pass01.logs')

In [5]:
# create user specific folders if needed
# ======================================
if not osp.exists(swarm_folder):
    os.makedirs(swarm_folder)
    print('++ INFO: New folder for swarm files created [%s]' % swarm_folder)
if not osp.exists(logdir_path):
    os.makedirs(logdir_path)
    print('++ INFO: New folder for log files created [%s]' % logdir_path)

In [6]:
# Open the file
swarm_file = open(swarm_path, "w")
# Log the date and time when the SWARM file is created
swarm_file.write('#Create Time: %s' % datetime.now().strftime("%d/%m/%Y %H:%M:%S"))
swarm_file.write('\n')
# Insert comment line with SWARM command
swarm_file.write('#swarm -f {swarm_path} -g 32 -t 32 --partition quick,norm --logdir {logdir_path}'.format(swarm_path=swarm_path,logdir_path=logdir_path))
swarm_file.write('\n')

# Insert one line per subject
for sbj,run in scan_list:
    run = run[-2:] + "_" + run[12:18]
    swarm_file.write("export SBJ={sbj} RUN={RUN}; sh {scripts_folder}/S04_TransformToMNI.pass01.sh".format(sbj=sbj, RUN=run, scripts_folder = SCRIPTS_DIR))
    swarm_file.write('\n')
swarm_file.close()

By the end of these jobs, we will have two new files in ```DATA_DIR/PrcsData/<SBJ>/preprocessed/func/```

* ```pb05_mni/<SCAN_ID>/rest2mni.nii.gz``` MNI Version of the motion corrected resting-state scan.
* ```pb05_mni/<SCAN_ID>/rest_mean_2mni.nii.gz``` MNI Version of the temporal mean of the file above. 

Becuase those files are very large (~2GB per scan), we decided to trim the corners of the files that contain no brain tissue. This required the following additional steps:

1. Create a common grid that would accomodate all scans

2. Cut the scans to be on that grid.

The following cells help us accomplish these two tasks

# 4. Compute common small size grid (only brain tissue)

In [11]:
command = """module load afni; \
             sh {PRJ_DIR}/code/fc_introspection/bash/S04_TransformToMNI.pass02.sh""".format(PRJ_DIR=PRJ_DIR)
output  = subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT)
print(output.strip().decode())

[+] Loading AFNI current-openmp  ... 
AFNI/current-openmp last updated  2023-01-22

++ 3dMean: AFNI version=AFNI_23.0.01 (Jan 18 2023) [64-bit]
++ 3dAutomask: AFNI version=AFNI_23.0.01 (Jan 18 2023) [64-bit]
++ Authored by: Emperor Zhark
++ Loading dataset all_mean.nii.gz
++ Forming automask
 + Fixed clip level = 268.126709
 + Used gradual clip level = 216.908188 .. 341.057709
 + Number voxels above clip level = 220587
 + Clustering voxels ...
 + Largest cluster has 219159 voxels
 + Clustering voxels ...
 + Largest cluster has 218471 voxels
 + Filled   534 voxels in small holes; now have 219005 voxels
 + Filled  2154 voxels in large holes; now have 221159 voxels
 + Clustering voxels ...
 + Largest cluster has 221156 voxels
 + Clustering non-brain voxels ...
 + Clustering voxels ...
 + Largest cluster has 681473 voxels
 + Mask now has 221156 voxels
++ 221156 voxels in the mask [out of 902629: 24.50%]
++ first   8 x-planes are zero [from R]
++ last   11 x-planes are zero [from L]
++ firs

>**NOTE**: Automask sometimes leaves a bit of skull on the left side of the brain. We will manually correct this error on all_mean.mask.boxed.nii.gz using AFNI.

# 5. Enforce new grid on files generated during step 3

In [12]:
#user specific folders
#=====================
swarm_path     = osp.join(swarm_folder,'S04_TransformToMNI.pass03.SWARM.sh')
logdir_path    = osp.join(logs_folder, 'S04_TransformToMNI.pass03.logs')

In [13]:
# create user specific folders if needed
# ======================================
if not osp.exists(swarm_folder):
    os.makedirs(swarm_folder)
    print('++ INFO: New folder for swarm files created [%s]' % swarm_folder)
if not osp.exists(logdir_path):
    os.makedirs(logdir_path)
    print('++ INFO: New folder for log files created [%s]' % logdir_path)

++ INFO: New folder for log files created [/data/SFIMJGC_Introspec/2023_fc_introspection/Logs.javiergc/S04_TransformToMNI.pass03.logs]


In [14]:
# Open the file
swarm_file = open(swarm_path, "w")
# Log the date and time when the SWARM file is created
swarm_file.write('#Create Time: %s' % datetime.now().strftime("%d/%m/%Y %H:%M:%S"))
swarm_file.write('\n')
# Insert comment line with SWARM command
swarm_file.write('#swarm -f {swarm_path} -g 32 -t 32 --partition quick,norm --logdir {logdir_path}'.format(swarm_path=swarm_path,logdir_path=logdir_path))
swarm_file.write('\n')

# Insert one line per subject
for sbj,run in scan_list:
    run = run[-2:] + "_" + run[12:18]
    swarm_file.write("export SBJ={sbj} RUN={RUN}; sh {scripts_folder}/S04_TransformToMNI.pass03.sh".format(sbj=sbj, RUN=run, scripts_folder = SCRIPTS_DIR))
    swarm_file.write('\n')
swarm_file.close()